<a href="https://colab.research.google.com/github/ldk7024/Machine_Learning_Study/blob/main/21_06_15_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 목표
- 네이버 영화리뷰 데이터셋을 이용해 긍정/부정 분류모델을 만들어보자
- TF IDF 방식을 적용해보자
- konlpy 한국어 형태소 분석기를 설치하고 활용해보자

### 데이터 수집
- 네이버 데이터를 로드
- label (0: 부정, 1: 긍정)

In [6]:
import pandas as pd
import numpy as np

In [9]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings_train.txt', delimiter='\t')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings_test.txt', delimiter='\t')
# delimiter: 파일에서 데이터 사이의 간격을 구분해주는 구분자 (설정하지 않으면 , 로 구분 )

In [13]:
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [14]:
df_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [12]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [15]:
df_train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [19]:
# train, test 데이터의 결측치가 있는 행을 불리언 인덱싱으로 확인해보세요
df_train[df_train['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [20]:
df_test[df_test['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [21]:
# 결측치가 있는 행을 제거
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [23]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- 문제/정답, 학습/평가 데이터 분리

In [27]:
# 문제
X_train = df_train['document']
y_train = df_train['label']
# 정답
X_test = df_test['document']
y_test = df_test['label']

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


### BOW 기법 (단어사전 만들기)
- counterVectorize (단순히 단어의 빈도만 파악)
- TF-IDF (단어의 빈도 + 단어가 등장하는 문서의 개수 둘다 파악)

In [29]:
# TF-IDF를 판단하기 위해 TfidVectorizer를 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
tf_idf_vect = TfidfVectorizer()
#  TfidfVectorizer의 디폴트값은 word (단어) 기준으로 토큰화를 진행

In [31]:
tf_idf_vect.fit(X_train[:3])
# TF-IDF 방식으로 단어사전 만들기 (학습)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [32]:
# vocabulary : 토큰화 된 단어와 인덱스의 라벨을 확인
tf_idf_vect.vocabulary_

{'가볍지': 0,
 '너무재밓었다그래서보는것을추천한다': 1,
 '더빙': 2,
 '목소리': 3,
 '않구나': 4,
 '오버연기조차': 5,
 '진짜': 6,
 '짜증나네요': 7,
 '초딩영화줄': 8,
 '포스터보고': 9}

- 영어에 맞춰져 있어서 한글 텍스트에서는 토큰화가 제대로 이루어지고 있지 않음

### 한글에 특화되어 있는 konply - Kkma(꼬꼬마) 사용하기

In [34]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 231kB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 450kB 27.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [36]:
# 한글을 형태소별로 잘 토큰화 할 수 있는 꼬꼬마 라이브러리를 불러오기
from konlpy.tag import Kkma

In [37]:
kkma = Kkma()

In [38]:
# nouns: 명사만 토큰화 시켜보자!
kkma.nouns(X_train[0])

['더빙', '목소리']

In [40]:
kkma.nouns('도지코인은 화성이 아니라 한강간다.')

['도지', '도지코인은', '코', '인은', '화성', '한강']

In [41]:
# 입력 문장이 들어왔을 때 명사만 토큰화 시키는 함수를 선언
def myTokenizer (text):
  return kkma.nouns(text)

In [42]:
# kkma 라이브러리에 TF-IDF 방식을 활용해보자
# tokenizer: 토큰의 단위를 지정, 디폴트 값은 word (단어)
tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer)

# 3개의 행에 대해서만 먼저 진행해보자!
tf_idf_vect.fit(X_train[:3])
tf_idf_vect.vocabulary_

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'더빙': 0,
 '딩': 1,
 '목소리': 2,
 '밓': 3,
 '보고': 4,
 '연기': 5,
 '영화': 6,
 '오버': 7,
 '오버연기': 8,
 '재': 9,
 '재밓': 10,
 '줄': 11,
 '초': 12,
 '초딩영화줄': 13,
 '추천': 14,
 '포스터': 15,
 '포스터보고': 16,
 '흠': 17}

### POS (Part Of Speech) Tagging 활용
- 문장을 토큰화 한 후 쪼개진 형태소에 품사를 부여하는 것

In [45]:
# 한글에서는 형용사와 동사에 감정이 많이 들어가 있음
data = '먹는다 먹다 먹었다 이쁘다 아름답다 사진 모자 첫사랑'
# morphs: 텍스트 (문장)을 형태소로 분리
kkma.morphs(data)

['먹',
 '는',
 '다',
 '먹',
 '다',
 '먹',
 '었',
 '다',
 '이쁘',
 '다',
 '아름답',
 '다',
 '사진',
 '모자',
 '첫사랑']

In [46]:
kkma.pos(data)

[('먹', 'VV'),
 ('는', 'EPT'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('었', 'EPT'),
 ('다', 'EFN'),
 ('이쁘', 'VA'),
 ('다', 'ECS'),
 ('아름답', 'VA'),
 ('다', 'ECS'),
 ('사진', 'NNG'),
 ('모자', 'NNG'),
 ('첫사랑', 'NNG')]

In [48]:
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [54]:
# 학습을 시키기 위해 데이터 프레임화
df= pd.DataFrame(kkma.pos(data), columns=['morph', 'tag'])
df

,morph,tag
0,먹,VV
1,는,EPT
2,다,ECS
3,먹,VV
4,다,ECS
5,먹,VV
6,었,EPT
7,다,EFN
8,이쁘,VA
9,다,ECS


In [55]:
# 데이터프레임에서 tag를 인덱스로 설정
df.set_index('tag', inplace=True)
# 동사(VV), 형용사(VA), 보통명사(NNG)로 인덱싱
df.loc[['VV', 'VA', 'NNG']]

,morph
tag,
VV,먹
VV,먹
VV,먹
VA,이쁘
VA,아름답
NNG,사진
NNG,모자
NNG,첫사랑


In [57]:
# 입력 텍스트에 대해서 pos tagging을 진행하여 VV,VA,NNG 값만 반환하는 함수를 선언
def myTokenizer2(text):
  # 텍스트가 들어오면 데이터프레임으로 변경 후 품사를 인덱스로 설정
  df= pd.DataFrame(kkma.pos(text), columns=['morph', 'tag'])
  df.set_index('tag', inplace=True)

  # VV이거나 VA이거나 NNG일때 그 값을 반환
  if('VV' in df.index) | ('VA' in df.index) |  ('NNG' in df.index):
    labels = ['VV', 'VA', 'NNG']
    # intersection() : 괄호안의 라벨들의 교집합까지 포함해서 배열로 변환
    return df.loc [df.index.intersection(labels)]['morph'].values

    ## VV, VA, NNG가 아니라면 필요없기 때문에 배열로반환
  else: 
    return[]

In [60]:
# 감성 분석에 필요한 동사, 형용사, 보통명사만 추출하는 TF-IDF 벡터라이저를 생성
tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer2)
tf_idf_vect.fit(X_train[:2])
tf_idf_vect.vocabulary_

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'가볍': 0,
 '더빙': 1,
 '목소리': 2,
 '보고': 3,
 '아': 4,
 '연기': 5,
 '영화': 6,
 '오버': 7,
 '줄': 8,
 '짜증나': 9,
 '포스터': 10,
 '흠': 11}

In [61]:
final_tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer2)
final_tf_idf_vect.fit(X_train[:3000])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function myTokenizer2 at 0x7f9d752e30e0>,
                use_idf=True, vocabulary=None)

In [63]:
# 동사, 형용사, 일반명사만 추출하여 만들어진 단어모음의 개수
len(final_tf_idf_vect.vocabulary_)

4743

In [64]:
# 토큰화된 단어를 기계가 이해할 수 있도록 벡터형태로 수치화 (transform)
X_train = final_tf_idf_vect.transform (X_train[:3000])
X_test = final_tf_idf_vect.transform (X_test[:3000])

In [65]:
from sklearn.linear_model import LogisticRegression

In [66]:
logi = LogisticRegression()
logi.fit(X_train, y_train[:3000])
logi.score(X_test, y_test[:3000])

0.7533333333333333